# Practica 1

## Parte B

### Ejercicio 1:

Instalamos el dataset "ml-100k" (ya esta incluido con la libreria scikit-surprise).

In [1]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505177 sha256=283b774b4686969eb3055d4891d9f56ec416df84e9e26aac51c0a90f017d3366
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
import os
import pandas as pd
import numpy as np

# Variable SEED, para que cada vez que se ejecute el código salgan los mismos resultados
SEED = 123

In [3]:
# Cargar movielens-100k dataset
from surprise import Dataset

data = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


### Ejercicio 2:

surprise tiene funcion **train_test_split** que acepta datos de entrada y se puede especificar que fraccion de datos vamos a usar para la evaluacion.

In [45]:
# Dividir data en particiones de entrenamiento (75%) y evaluacion (25%)
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25, random_state=SEED)

### Ejercicio 3/4:

En primer caso, utilizamos KNNBasic con la metrica de similtud pearson para realizar filtrado colaborativo basado en vecinos. KNN es un clasificador de aprendizaje supervisado no paramétrico que utiliza la proximidad para realizar predicciones sobre la agrupación de un punto de datos individual.

Primer caso es basado en usarios.

In [46]:
# Filtrado KNNBasic basado en usarios
from surprise import KNNBasic
from surprise.model_selection.validation import cross_validate

knn_basic_user = KNNBasic(sim_options={"name": "pearson", "user_based": True}, random_state=SEED)
knn_basic_user.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


En segundo caso, todavia utilizamos KNNBasic sino en este caso para filtrado basado en productos.

In [47]:
# Filtrado KNNBasic basado en productos
from surprise import KNNBasic

knn_basic_product = KNNBasic(sim_options={"name": "pearson", "user_based": False}, random_state=SEED)
knn_basic_product.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


En la otra parte, utilizamos SVD y NMF para realizar filtrado colaborativo basado en modelos usando factorización de matrices.

En primer caso, utilizamos SVD. La idea de este algoritmo es hacer operaciones de algebra lineal - factorización de una matriz real o compleja en una rotación, seguida de un reescalado seguido de otra rotación.

In [48]:
# Filtrado colaborativo SVD
from surprise import SVD

svd = SVD(random_state=SEED)
svd.fit(trainset)

En segundo caso, utilizamos NMF. La factorización de matrices no negativas es un algoritmo en el que una matriz se factoriza en otras dos matrices con la propiedad de que las tres matrices no tienen elementos negativos. Esta no negatividad hace que las matrices resultantes sean más fáciles de verificar.

In [49]:
# Filtrado colaborativo NMF
from surprise import NMF

nmf = NMF(random_state=SEED)
nmf.fit(trainset)

### Ejercicio 5:

Cuando tenemos todos algoritmos entrenados, utlizamos funcion **test** con cada de los algoritmos y le pasamos datos de evaluacion. Pero antes de eso, es importante asegurar que datos de testset son limpios y utiles.

In [50]:
# Hacer predicciones para conjunto de evaluacion con cada algoritmo
from surprise import accuracy

predictions_user = knn_basic_user.test(testset)
predictions_product = knn_basic_product.test(testset)
predictions_svd = svd.test(testset)
predictions_nmf = nmf.test(testset)

# Mostrar 5 resultados de cada predicciones
N = 5

res_user = predictions_user[:N]
res_product = predictions_product[:N]
res_svd = predictions_svd[:N]
res_nmf = predictions_nmf[:N]

from pprint import pprint

print("FILTRADO BASADO EN USUARIOS")
pprint(res_user)
print()
print("FILTRADO BASADO EN PRODUCTOS")
pprint(res_product)
print()
print("FILTRADO COLLABORATIVO SVD")
pprint(res_svd)
print()
print("FILTRADO COLLABORATIVO NMF")
pprint(res_nmf)
print()

FILTRADO BASADO EN USUARIOS
[Prediction(uid='798', iid='480', r_ui=3.0, est=4.220708572354324, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='296', iid='462', r_ui=4.0, est=3.860840081088127, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='653', iid='258', r_ui=3.0, est=3.88914154911376, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='883', iid='239', r_ui=3.0, est=3.5412030693605168, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='272', iid='187', r_ui=5.0, est=4.255937339224406, details={'actual_k': 40, 'was_impossible': False})]

FILTRADO BASADO EN PRODUCTOS
[Prediction(uid='798', iid='480', r_ui=3.0, est=3.2132150484522715, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='296', iid='462', r_ui=4.0, est=4.292899057345213, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='653', iid='258', r_ui=3.0, est=2.2120393116387986, details={'actual_k': 40, 'was_impossible

En cada de las predicciones, podemos ver atributos "est" y "r_ui" que nos interesan mas. Est nos muestra la valor prediccionada (estimada) comparando con la referencia (r_ui). Estos datos nos dan la primera impresion de la calidad de predicciones de cada algoritmo.

### Ejercicio 6:

En esta parte obtenemos difetentes metrics estadisticos para evaluar cada modelo y hacer conclusiones. Empezamos por RMSE, usando la funcion **accuracy** de surprise.

In [51]:
from surprise import accuracy

# Evaluacion RMSE
rmse_user = accuracy.rmse(predictions_user, verbose=True)
rmse_product = accuracy.rmse(predictions_product, verbose=True)
rmse_svd = accuracy.rmse(predictions_svd, verbose=True)
rmse_nmf = accuracy.rmse(predictions_nmf, verbose=True)

RMSE: 1.0116
RMSE: 1.0361
RMSE: 0.9322
RMSE: 0.9655


La funcion siguiente hace numerosas operaciones para obtener precision:
1. agrupa todas las predicciones por usario (para que si queremos introducir precision@k, lo podemos hacer mas facilmente);
2. para cada usario, ordena los items por el rating prediccionado;
3. crea un vector de valores verdaderos en el que cada item se considera relevante si su rating real es mayor a 4;
4. calcula la precision usando precision_score de sklrean para cada usuario.

Precision es util porque nos muestra la fraccion de instancias relevantes entre las instancias recuperadas.

In [52]:
# Evaluacion precision usando precision_score
import numpy as np
from sklearn.metrics import precision_score
from collections import defaultdict

def precision(predictions, threshold=4.0):
    # grupa predicciones por usario
    user_preds = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_preds[uid].append((iid, true_r, est))

    precisions = []
    for uid, preds in user_preds.items():
        # ordena predicciones
        preds_sorted = sorted(preds, key=lambda x: x[2], reverse=True)
        n = len(preds_sorted)

        # crea vector binario de prediccion (relevante = 1, irrelevante = 0).
        y_true = np.array([1 if true_r >= threshold else 0 for (_, true_r, _) in preds_sorted])
        y_pred = np.array([1 if est >= threshold else 0 for (_, _, est) in preds_sorted])

        # calcula precision
        prec = precision_score(y_true, y_pred, zero_division=0)
        precisions.append(prec)

    return np.mean(precisions)

In [53]:
# obtenemos los valores de precision
precision_user = precision(predictions_user)
precision_product = precision(predictions_product)
precision_svd = precision(predictions_svd)
precision_nmf = precision(predictions_nmf)

print("Precision@10 (basado en usario):", precision_user)
print("Precision@10 (basado en productos):", precision_product)
print("Precision@10 (SVD):", precision_svd)
print("Precision@10 (NMF):", precision_nmf)

Precision@10 (basado en usario): 0.6695960573997044
Precision@10 (basado en productos): 0.33585736148861295
Precision@10 (SVD): 0.6344181404352349
Precision@10 (NMF): 0.6268556199863577


En la funcion **recall** la preparacion es parecida a la de **precision**, sino al final utilizamos recall_score para obtener recall. La metrica de recall muestra la fraccion de instancias relevantes que se recuperaron.

In [54]:
import numpy as np
from collections import defaultdict
from sklearn.metrics import recall_score

# Evaluacion recall usando recall_score
def recall(predictions, threshold=4.0):
    # grupa predicciones por usario
    user_preds = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_preds[uid].append((iid, true_r, est))

    recalls = []
    for uid, preds in user_preds.items():
        # ordena predicciones
        preds_sorted = sorted(preds, key=lambda x: x[2], reverse=True)
        n = len(preds_sorted)

        # crea vector binario de prediccion (relevant = 1, irrelevante = 0).
        y_true = np.array([1 if true_r > threshold else 0 for (_, true_r, _) in preds_sorted])
        y_pred = np.array([1 if est > threshold else 0 for (_, _, est) in preds_sorted])

        # calcula recall
        rec = recall_score(y_true, y_pred, zero_division=0)
        recalls.append(rec)

    return np.mean(recalls)

In [55]:
# obtenemos los valores de recall
recall_user = recall(predictions_user)
recall_product = recall(predictions_product)
recall_svd = recall(predictions_svd)
recall_nmf = recall(predictions_nmf)

print("Recall@10 (basado en usario):", recall_user)
print("Recall@10 (basado en productos):", recall_product)
print("Recall@10 (SVD):", recall_svd)
print("Recall@10 (NMF):", recall_nmf)

Recall@10 (basado en usario): 0.38047194902917825
Recall@10 (basado en productos): 0.20634545020313136
Recall@10 (SVD): 0.3798508766391291
Recall@10 (NMF): 0.3669924892838016


La funcion de NDCG hace las siguientes operaciones:
1. agrupa todas las predicciones por usario;
2. ordena las predicciones por rating estimado descendiente;
3. crea un vector binario de relevancia (1 si el rating real es mayor que el 4, 0 si no);
4. crea un vector con los ratings estimados correspondientes a esos items;
5. calcula el NDCG para cada usuario usando estos vectores.

La metrica NDCG muestra la relevancia de los elementos devueltos por el modelo comparado con la relevancia del elemento que devolvería un modelo “ideal” hipotético.

In [56]:
K = 10

import numpy as np
from collections import defaultdict
from sklearn.metrics import ndcg_score

# Evaluacion recall usando ndcg_score
def ndcg(predictions, k=10, threshold=4.0):
    # grupa predicciones por usario
    user_preds = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_preds[uid].append((iid, true_r, est))

    ndcg_list = []
    for uid, items in user_preds.items():
        # ordena predicciones
        items_sorted = sorted(items, key=lambda x: x[2], reverse=True)
        n_items = len(items_sorted)

        # crea vector binario de prediccion y vector de valores estimados
        y_true = np.array([[1 if true_r > threshold else 0 for (_, true_r, _) in items_sorted]])
        y_scores = np.array([[est for (_, _, est) in items_sorted]])

        # si hay solo un elemento, agrega un elemento fake (0 relevancia y 0 puntuación)
        if n_items < 2:
            pad = 2 - n_items
            y_true = np.concatenate([y_true, np.zeros((1, pad))], axis=1)
            y_scores = np.concatenate([y_scores, np.zeros((1, pad))], axis=1)
            current_k = 2
        else:
            current_k = n_items

        # calcula NDCG
        ndcg_value = ndcg_score(y_true, y_scores, k=current_k)
        ndcg_list.append(ndcg_value)

    return np.mean(ndcg_list)

In [57]:
ndcg_user = ndcg(predictions_user, k=K)
ndcg_product = ndcg(predictions_product, k=K)
ndcg_svd = ndcg(predictions_svd, k=K)
ndcg_nmf = ndcg(predictions_nmf, k=K)

print("NDCG@10 (basado en usario):", ndcg_user)
print("NDCG@10 (basado en productos):", ndcg_product)
print("NDCG@10 (SVD):", ndcg_svd)
print("NDCG@10 (NMF):", ndcg_nmf)

NDCG@10 (basado en usario): 0.6251818033312211
NDCG@10 (basado en productos): 0.5563687562620963
NDCG@10 (SVD): 0.6242711442075215
NDCG@10 (NMF): 0.6065251748493437


In [58]:
# crear tabla consolidada de RMSE, precision, recall, ndcg
import pandas as pd

data = {'Algoritmo': ['Basado en Usuarios', 'Basado en Productos', 'SVD', 'NMF'],
  'RMSE': [
    rmse_user,
    rmse_product,
    rmse_svd,
    rmse_nmf
  ],
  'Precision': [
    precision_user,
    precision_product,
    precision_svd,
    precision_nmf
  ],
  'Recall': [
    recall_user,
    recall_product,
    recall_svd,
    recall_nmf
  ],
  'NDCG': [
    ndcg_user,
    ndcg_product,
    ndcg_svd,
    ndcg_nmf
  ]
}

df = pd.DataFrame(data)
df

,Algoritmo,RMSE,Precision,Recall,NDCG
0,Basado en Usuarios,1.011588,0.669596,0.380472,0.625182
1,Basado en Productos,1.036061,0.335857,0.206345,0.556369
2,SVD,0.932241,0.634418,0.379851,0.624271
3,NMF,0.965540,0.626856,0.366992,0.606525


Análisis de los resultados:

Basado en Usuarios
- En general, sobresale en las metricas de recomendacion (precision, recall y NDCG), aunque no es el mejor en RMSE.

Basado en Productos
- En general, funciona algo peor que el basado en usuarios en todas las metricas.

SVD
- Es buena para predecir la calificacion numerica (menor RMSE), pero su ranking (NDCG) no es el mas alto.

NMF
- En general, es el metodo que rinde peor en estas metricas comparado con los otros tres.

Conclusiones:

Si la prioridad es la exactitud numerica de las predicciones (que el sistema acierte con la calificacion lo mas cerca posible a la real), el mejor es SVD porque tiene el RMSE mas bajo.

Si la prioridad es la calidad de la recomendacion en terminos de relevancia (cantidad de items que realmente le gustan al usario), en este caso KNNBasic basado en Usuarios es superior (mejor precisión, recall y NDCG).

Dado que en la mayoría de escenarios de recomendacion se valoran mas las metricas de relevancia (precision, recall, NDCG) que la exactitud absoluta de, el metodo basado en usuarios puede ser la mejor eleccion global entre los algoritmos presentados.